<a href="https://colab.research.google.com/github/rachelinaputri/data-mini-project/blob/main/10_Fold_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import make_scorer, accuracy_score, recall_score, f1_score, confusion_matrix

def specificity_score(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=np.unique(y_true)).ravel()
    return tn / (tn + fp)

# Function to preprocess data: split columns, convert data types
def preprocess_data(df):
    df = df[0].str.split(';', expand=True)
    df.columns = df.iloc[0]  # Use the first row as column headers
    df = df.drop(df.index[0])
    features = df.iloc[:, :-1].apply(pd.to_numeric)  # Convert feature columns to numeric
    target = df.iloc[:, -1].astype('category')  # Convert class label to categorical
    return features, target

# Load the data files
cpa_file_paths = ['CPA1.csv', 'CPA2.csv', 'CPA3.csv']
cpa_dataframes = [pd.read_csv(file, header=None) for file in cpa_file_paths]

# Preprocess the data
cpa_features_targets = [preprocess_data(df) for df in cpa_dataframes]

# Define classifiers
svm_classifier = SVC(kernel='linear', random_state=42)
knn_classifier = KNeighborsClassifier(n_neighbors=5)

# Scoring metrics
scoring_metrics = {
    'accuracy': make_scorer(accuracy_score),
    'recall': make_scorer(recall_score, average='weighted'),  # Sensitivity
    'specificity': make_scorer(specificity_score),  # Specificity
    'f1': make_scorer(f1_score, average='weighted')
}

# Perform classification and evaluation
results = []
for features, target in cpa_features_targets:
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    # SVM results
    svm_results = cross_validate(svm_classifier, features_scaled, target, cv=cv, scoring=scoring_metrics)
    svm_summary = {metric: scores.mean() for metric, scores in svm_results.items() if 'test_' in metric}

    # K-NN results
    knn_results = cross_validate(knn_classifier, features_scaled, target, cv=cv, scoring=scoring_metrics)
    knn_summary = {metric: scores.mean() for metric, scores in knn_results.items() if 'test_' in metric}

    results.append((svm_summary, knn_summary))

# Display results
for result in results:
    print("SVM Results:", result[0])
    print("K-NN Results:", result[1])


SVM Results: {'test_accuracy': 0.8150000000000001, 'test_recall': 0.8150000000000001, 'test_specificity': 0.8300000000000001, 'test_f1': 0.8142819169134959}
K-NN Results: {'test_accuracy': 0.745, 'test_recall': 0.745, 'test_specificity': 0.76, 'test_f1': 0.7430026123201517}
SVM Results: {'test_accuracy': 0.705, 'test_recall': 0.705, 'test_specificity': 0.6499999999999999, 'test_f1': 0.701172834434377}
K-NN Results: {'test_accuracy': 0.735, 'test_recall': 0.735, 'test_specificity': 0.8100000000000002, 'test_f1': 0.7277787475682211}
SVM Results: {'test_accuracy': 0.6900000000000001, 'test_recall': 0.6900000000000001, 'test_specificity': 0.9099999999999999, 'test_f1': 0.6448142198223981}
K-NN Results: {'test_accuracy': 0.79, 'test_recall': 0.79, 'test_specificity': 0.9099999999999999, 'test_f1': 0.7781882304912732}
